<a href="https://colab.research.google.com/github/JayJay95/JavaByteCodeGenerator/blob/master/TestingVulnerabilityClassifier_SimpleCNNwithDataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd /content/drive/My\ Drive/After4thYear/Belfast/MSc\ Cybersec/Research\ Project/Colab Notebooks

In [0]:
import os
import re
import fnmatch
import argparse
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import test_cases
from copy import deepcopy
import pdb
import time
import matplotlib.pyplot as plt
import random
# import sys
# sys.path.append('/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/')
# from lr_finder import LRFinder


In [0]:
clean_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Clean_Opseq'
vuln_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Vuln_Opseq'

In [0]:
parser = argparse.ArgumentParser(description='Vulnerability Classifier')
# parser.add_argument('--max_opcode_seq_len', action='store', type=int, help='use different versions of network', default=8192)
# parser.add_argument('--min_opcode_seq_len', action='store', type=int, help='use different versions of network', default=32)
parser.add_argument('--lr', action='store', type=float, help='use different versions of network', default=1e-5)
parser.add_argument('--epochs', action='store', type=int, help='use different versions of network', default=1)
opt = parser.parse_args('')
print(opt)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
def find_files(files, dirs=[], extensions=[]): # recursively find files in directories
    new_dirs = []
    for d in dirs:
        try:
            new_dirs += [ os.path.join(d, f) for f in os.listdir(d)] # check in all directories except testcasesupport 
        except OSError:
            if os.path.splitext(d)[1] in extensions:
                files.append(d)

    if new_dirs:
        find_files(files, new_dirs, extensions)
    else:
        return  

def read_file(filename):
    opcode_count = 0
    line_list = []
    with open(filename, mode='rt', encoding='utf8') as f:
        content = f.readlines()        
    for line in content:
        opcode_seq = []     
        for c in range(0, len(line) - 1, 2):
            #print(line[c:(c+2)],int(line[c:(c+2)], 16))
            opcode_seq.append(int(line[c:(c+2)], 16) + 1) # add one here so that the zero'th embedding is reserved for 'blank' i.e. no instruction whatsoever not even no-op			
            opcode_count += 1
            # to save training time we only read 
            # the first opt.max_opcode_seq_len opcodes of each file
            
            # if opcode_count >= opt.max_opcode_seq_len:
            #     return opcode_seq
        line_list.append(opcode_seq)
    return line_list

def read_dataset():
    vuln = []
    clean = []
    clean_opseq_files = []
    vuln_opseq_files = []
    # min_file_len = opt.min_opcode_seq_len #ignore opcode seq files shorter than this
    find_files(clean_opseq_files, dirs=[clean_opseq_folder], extensions=['.clean'])
    for clean_file_pathname in clean_opseq_files:
        tmp = read_file(clean_file_pathname)
        # if len(tmp) >= min_file_len:
        clean.append(tmp)
    
    find_files(vuln_opseq_files, dirs=[vuln_opseq_folder], extensions=['.vuln'])
    for vuln_file_pathname in vuln_opseq_files:
        tmp = read_file(vuln_file_pathname)
        # if len(tmp) >= min_file_len:
        vuln.append(tmp)
    
    # flatten vuln and clean lists
    new_vuln = []
    for x in vuln:
        for y in x:
            new_vuln.append(y)
    
    new_clean = []
    for x in clean:
        for y in x:
            new_clean.append(y)
    return new_vuln, new_clean

In [0]:
def split_dataset(vuln, clean):
    #split the dataset into train, val, test sets
    #return the concatenated dataset and
    #indicies pointing to the train,val,test samples

    vuln_label = 0
    clean_label = 1
    dataset = deepcopy(clean) + deepcopy(vuln) # concatenate original clean and original vuln samples
    trial_dataset = clean + vuln
    
    #pad with zeroes to make all sequences a standard length of 1000
    for list in trial_dataset:
        if len(list) < 1000:
            list.extend([0] * (1000- len(list)))
    
    final_vuln_list = []
    for vuln_list in vuln:
        inner_vuln_list = []
        inner_vuln_list.append(vuln_list)
        inner_vuln_list.append(vuln_label)
        final_vuln_list.append(inner_vuln_list)
             
    final_clean_list = []
    for clean_list in clean:
        inner_clean_list = []
        inner_clean_list.append(clean_list)
        inner_clean_list.append(clean_label)
        final_clean_list.append(inner_clean_list)
    
    # split vuln samples randomly
    first_vuln_split = int(0.8 * len(final_vuln_list))
    second_vuln_split = int(0.1 * len(final_vuln_list))
    third_vuln_split = int(len(final_vuln_list) - (first_vuln_split + second_vuln_split))
    vuln_training_dataset, vuln_validation_dataset, vuln_testing_dataset = torch.utils.data.random_split(final_vuln_list, [first_vuln_split, second_vuln_split, third_vuln_split])

    # split clean samples randomly
    first_clean_split = int(0.8 * len(final_clean_list))
    second_clean_split = int(0.1*len(final_clean_list))
    third_clean_split = int(len(final_clean_list) - (first_clean_split + second_clean_split))
    clean_training_dataset, clean_validation_dataset, clean_testing_dataset = torch.utils.data.random_split(final_clean_list, [first_clean_split, second_clean_split, third_clean_split])

    # merge both vuln & clean training sets
    training_set = clean_training_dataset + vuln_training_dataset

    # get indices and labels from training set
    train_inds=[]
    train_labels=[]
    training_counter = 0
    
    for list in training_set:
        train_inds.append(training_counter)
        training_counter+=1
        train_labels.append(list[1])
        
    # merge both vuln & clean validation sets
    validation_set = clean_validation_dataset + vuln_validation_dataset

    # get indices and labels from validation set
    val_inds = []
    val_labels = []
    training_set_length = len(training_set)
    for list in validation_set:
        val_inds.append(training_set_length)
        training_set_length += 1
        val_labels.append(list[1])

    # merge both vuln & clean testing sets
    testing_set = clean_testing_dataset + vuln_testing_dataset

    # get indices and labels from testing set
    test_inds = []
    test_labels = []
    validation_set_length = len(training_set) + len(validation_set)
    for list in testing_set:
        test_inds.append(validation_set_length)
        validation_set_length += 1
        test_labels.append(list[1])
#     pdb.set_trace()
    return trial_dataset, training_set, validation_set, testing_set


In [0]:
def evaluate_network_performance(predictions, ground_truth):
    # given an array with the predicted values and the correct values
    # calculate the precision, recall and f-score
    cm = get_confusion_matrix(predictions, ground_truth)
    TP = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TN = cm[1][1]

    precision = TP/(TP + FP + 1e-6)
    recall = TP/(TP + FN + 1e-6)
    classification_accuracy = (TP+TN) / (TP + TN + FP + FN + 1e-6)
    f_score = 2 * ((precision*recall)/(precision + recall))

    return precision, recall, f_score, classification_accuracy

In [0]:
class VulnerabilityDetectorNetwork(nn.Module):
    def __init__(self):
        super(VulnerabilityDetectorNetwork, self).__init__()

        self.num_of_embeddings = 205
        self.embedding_dimension = 8
        self.channels_in = 1
        self.channels_out = 64
        self.channels_out_two = 128
        self.hidden_nodes = 16
        self.kernel_height_dimension = 8
        self.kernel_width_dimension = 8
        self.padding_height_dimension = 0
        self.padding_width_dimension = 0
        self.features_out = 2

        self.emb1 = nn.Embedding(self.num_of_embeddings, self.embedding_dimension)
        self.conv1 = nn.Conv2d(self.channels_in, self.channels_out, kernel_size=(self.kernel_height_dimension, self.kernel_width_dimension),
            padding=(self.padding_height_dimension, self.padding_width_dimension))
#         self.conv2 = nn.Conv2d(self.channels_out, self.channels_out_two, kernel_size=(1,1),
#             padding=(self.padding_height_dimension, self.padding_width_dimension))
#         self.batchNorm = nn.BatchNorm2d(64)
        self.lin1 = nn.Linear(self.channels_out, self.hidden_nodes)
        self.lin2 = nn.Linear(self.hidden_nodes, self.features_out)
        self.drop_out = nn.Dropout(0.5)
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
#       print(x.shape)
        x = self.emb1(x)
#         print(x.shape)        
        x = x.unsqueeze(1)
#         x = x.unsqueeze(0)
#         print(x.shape)
        x = self.conv1(x)
#         print(x.shape)
#         x = self.batchNorm(x)
#         print (x.shape)
        x = F.relu(x)
#         print(x.shape)
#         x = self.conv2(x)
# #         print(x.shape)
# # #         x = self.batchNorm(x)
# # #         print (x.shape)
#         x = F.relu(x)
# #         print(x.shape)
        x = torch.max(x,2)[0]
#         print(x.shape)
        x = self.drop_out(x)
#         print(x.shape)
        x = x.squeeze(2)
#         print(x.shape)
        x = self.lin1(x)
#         print(x.shape)
        x = self.lin2(x)
#         print(x.shape)
#         x = self.softmax(x)
        return x

In [0]:
def get_confusion_matrix(preds, truth):
    flat_truth = torch.stack(truth).flatten()
    flat_preds = torch.stack(preds).flatten()
    K = len(np.unique(flat_truth)) # Number of classes 
    result = np.zeros((K, K))
    for i in range(len(flat_truth)):
        result[flat_preds[i]][flat_truth[i]] += 1
    confusion_matrix = result
    print(confusion_matrix)
    return confusion_matrix

In [0]:
# def test_network(net, tvtLoader):
    
#     criterion = nn.CrossEntropyLoss()

#     test_loss = 0
#     predictions = []
#     labels = []

#     with torch.no_grad():
# #         for i in range(len(inds)):
#         for i, (opsqs, lbls) in enumerate(tvtLoader):
#             net.eval()
#             opsqs = torch.stack(opsqs).flatten()
#             opsqs_tensor = opsqs.type(torch.LongTensor)
#             lbls = lbls.type(torch.LongTensor)
            
# #             output = net(opsqs_tensor.unsqueeze(0))
#             opsqs_tensor = opsqs_tensor.view(-1, 1000)
#             output = net(opsqs_tensor)
# #             output = net(opsqs_tensor.view(1, -1))
#             val,idx = torch.max(output.data,1) # max pool - max over the rows
# #             predictions.append(idx.item())
#             predictions.append(idx)
#             labels.append(lbls)

#             test_loss = criterion(output, lbls).item()

#         precision, recall, f_score, classification_accuracy = evaluate_network_performance(predictions, labels)

#     return precision, recall, f_score, classification_accuracy

def test_network(net, tvtLoader):
    
    criterion = nn.CrossEntropyLoss()

    test_loss = 0
    predictions = []
    labels = []

    with torch.no_grad():
#         for i in range(len(inds)):
        for i, (opsqs, lbls) in enumerate(tvtLoader):
            net.eval()
            opsqs = torch.stack(opsqs).flatten()
            opsqs_tensor = opsqs.type(torch.LongTensor)
            lbls = lbls.type(torch.LongTensor)
            output = net(opsqs_tensor.view(-1, 1000))
            val,idx = torch.max(output.data,1) # max pool - max over the rows
#             predictions.append(idx.item())
            predictions.append(idx)
            labels.append(lbls)

            test_loss = criterion(output, lbls).item()

        precision, recall, f_score, classification_accuracy = evaluate_network_performance(predictions, labels)

    return precision, recall, f_score, classification_accuracy


In [0]:
# def train_network(net, trainLoader, valLoader):
#     epochs_list = []
#     train_acc_list = []
#     val_acc_list = []
    
#     optimizer = optim.Adam(net.parameters(), lr=opt.lr)
#     criterion = nn.CrossEntropyLoss()

#     num_training_epochs = opt.epochs


#     for e in range(num_training_epochs):
#         running_loss = 0

#         for i, (opseqs, labels) in enumerate(trainLoader):
#             net.train() # set network into training mode
#             optimizer.zero_grad() #reset the optimizer before every loop
            
#             opseqs = torch.stack(opseqs).flatten()
#             opseqs_tensor = opseqs.type(torch.LongTensor)
#             labels = labels.type(torch.LongTensor)
#             opseqs_tensor = opseqs_tensor.view(-1, 1000)
# #             pdb.set_trace()
#             net_output = net(opseqs_tensor)
# #             net_output = net(opseqs_tensor.view(1, -1)) # forward pass of current training sample through the network
# #             net_output = net(opseqs_tensor.unsqueeze(0))
#             labels = Variable(labels)
#             loss = criterion(net_output, labels) # how close network is to correct answer
#             running_loss += loss.item()
            
#             loss.backward() # back propagation step for whole net
#             optimizer.step() # call optimizer to update network's parameters

#         epoch_loss = running_loss/len(trainLoader)
#         epochs_list.append(e)

#         print('iteration ', e, 'loss ', epoch_loss)
#         print('          prec rec  f1   acc')

#         precision, recall, f_score, classification_accuracy = test_network(net, trainLoader)
#         val_acc_list.append(classification_accuracy)
#         print('train set',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))
        
#         precision, recall, f_score, classification_accuracy = test_network(net, valLoader)
#         train_acc_list.append(classification_accuracy)
#         print('val set  ',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))

#         print()

#     return net, epochs_list, train_acc_list, val_acc_list

def train_network(net, trainLoader, valLoader):
    epochs_list = []
    train_acc_list = []
    val_acc_list = []
    loss_list = []
    
    optimizer = optim.Adam(net.parameters(), lr=opt.lr)
    criterion = nn.CrossEntropyLoss()

    num_training_epochs = opt.epochs

    for e in range(num_training_epochs):
        running_loss = 0

        for i, (opseqs, labels) in enumerate(trainLoader):
            net.train() # set network into training mode
            optimizer.zero_grad() #reset the optimizer before every loop
            
            opseqs = torch.stack(opseqs).flatten()
            opseqs_tensor = opseqs.type(torch.LongTensor)
            labels = labels.type(torch.LongTensor)
            
            net_output = net(opseqs_tensor.view(-1, 1000)) # forward pass of current training sample through the network
#           net_output = net(opseqs_tensor.unsqueeze(0))
            loss = criterion(net_output, labels) # how close network is to correct answer
            running_loss += loss.item()
            
            loss.backward() # back propagation step for whole net
            optimizer.step() # call optimizer to update network's parameters

        epoch_loss = running_loss/len(trainLoader)
        epochs_list.append(e)
        loss_list.append(epoch_loss)
        
        print('iteration ', e, ' loss ', epoch_loss)
        print('          prec rec  f1   acc')
         
        precision, recall, f_score, classification_accuracy = test_network(net, trainLoader)
        train_acc_list.append(classification_accuracy)
        print('train set',"{0:.10f}".format(precision),"{0:.10f}".format(recall),"{0:.10f}".format(f_score),"{0:.10f}".format(classification_accuracy))
        
        precision, recall, f_score, classification_accuracy = test_network(net, valLoader)
        val_acc_list.append(classification_accuracy)
        print('val set  ',"{0:.10f}".format(precision),"{0:.10f}".format(recall),"{0:.10f}".format(f_score),"{0:.10f}".format(classification_accuracy))

        print()

    return net, epochs_list, train_acc_list, val_acc_list, loss_list



In [0]:
def createLoaders(train_set, val_set, test_set):
    trainLoader = torch.utils.data.DataLoader(train_set, batch_size = 1, shuffle = True, drop_last = True, pin_memory = True)
    valLoader = torch.utils.data.DataLoader(val_set, batch_size = 1, shuffle = True, drop_last = True, pin_memory = True)
    testLoader = torch.utils.data.DataLoader(test_set, batch_size = 1, shuffle = False, drop_last = True, pin_memory = True)
    return trainLoader, valLoader, testLoader


In [0]:
def drawEpochsAccuracyGraph(x_epochs, y_train, y_val):
    plt.plot(x_epochs, y_train, label = "Train")
    plt.plot(x_epochs, y_val, label = "Val")

    # naming the axes
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    # title
    plt.title('Epochs vs Accuracy')
    plt.show()


In [0]:
def drawEpochsLossGraph(x_epochs, y_loss):
    plt.plot(x_epochs, y_loss, label = "Loss")
    # naming the axes
    plt.xlabel('Epochs')
    plt.ylabel('Loss (%)')
    plt.legend()

    # title
    plt.title('Epochs vs Loss')
    plt.show()


In [0]:
# realistic_clean_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Realistic_Clean_Opseq'
# realistic_vuln_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Realistic_Vuln_Opseq'

In [0]:
# def read_realistic_dataset():
#     vuln = []
#     clean = []
#     clean_opseq_files = []
#     vuln_opseq_files = []
#     # min_file_len = opt.min_opcode_seq_len #ignore opcode seq files shorter than this
#     find_files(clean_opseq_files, dirs=[realistic_clean_opseq_folder], extensions=['.clean'])
#     for clean_file_pathname in clean_opseq_files:
#         tmp = read_file(clean_file_pathname)
#         # if len(tmp) >= min_file_len:
#         clean.append(tmp)
    
#     find_files(vuln_opseq_files, dirs=[realistic_vuln_opseq_folder], extensions=['.vuln'])
#     for vuln_file_pathname in vuln_opseq_files:
#         tmp = read_file(vuln_file_pathname)
#         # if len(tmp) >= min_file_len:
#         vuln.append(tmp)
    
#     # flatten vuln and clean lists
#     new_vuln = []
#     for x in vuln:
#         for y in x:
#             new_vuln.append(y)
    
#     new_clean = []
#     for x in clean:
#         for y in x:
#             new_clean.append(y)
#     return new_vuln, new_clean

In [0]:
# def testRealisticSet():
#   vuln, clean = read_realistic_dataset()
#   vuln_label = 0
#   clean_label = 1
#   dataset = deepcopy(clean) + deepcopy(vuln) # concatenate original clean and original vuln samples
#   trial_dataset = clean + vuln

#   #pad with zeroes to make all sequences a standard length of 1000
#   for list in trial_dataset:
#       if len(list) < 1000:
#           list.extend([0] * (1000- len(list)))

#   final_vuln_list = []
#   for vuln_list in vuln:
#       inner_vuln_list = []
#       inner_vuln_list.append(vuln_list)
#       inner_vuln_list.append(vuln_label)
#       final_vuln_list.append(inner_vuln_list)

#   final_clean_list = []
#   for clean_list in clean:
#       inner_clean_list = []
#       inner_clean_list.append(clean_list)
#       inner_clean_list.append(clean_label)
#       final_clean_list.append(inner_clean_list)
  
#   test_realistic_set = final_clean_list + final_vuln_list
#   testRealisticLoader = torch.utils.data.DataLoader(test_realistic_set, batch_size = 1, shuffle = False, drop_last = True) 
#   return testRealisticLoader

In [0]:
def create_detection_set():
  detection_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Detection_Opseq'
  detection_set = []
  detection_opseq_files = []
  find_files(detection_opseq_files, dirs=[detection_opseq_folder], extensions=['.txt'])
  for detection_file_pathname in detection_opseq_files:
      tmp = read_file(detection_file_pathname)
      detection_set.append(tmp)

  # flatten list
  new_detection_set = []
  for x in detection_set:
      for y in x:
          new_detection_set.append(y)
  
  #pad with zeroes to make all sequences a standard length of 1000
  for list in new_detection_set:
      if len(list) < 1000:
          list.extend([0] * (1000- len(list)))
  testDetectionLoader = torch.utils.data.DataLoader(new_detection_set, batch_size = 1, shuffle = False, drop_last = True)       
  return testDetectionLoader

In [0]:
def test_detection(net, tvtLoader):
  predictions = []
  
  with torch.no_grad():
    for i, (opsqs) in enumerate(tvtLoader):
        net.eval()
        opsqs = torch.stack(opsqs).flatten()
        opsqs_tensor = opsqs.type(torch.LongTensor)
        output = net(opsqs_tensor.view(-1, 1000))
        val,idx = torch.max(output.data,1) # max pool - max over the rows
        predictions.append(idx.item())
        pdb.set_trace()
  for pred_item in predictions:
    if (pred_item == 0):
      print("Alert! Alert! Vulnerability detected")
      print (pred_item)
    else: 
      print("Hurray it's clean!")
      print (pred_item)


In [0]:
def main():
    # create_clean_opseq_files() # should only run the first time this code is ran
    # print('Created clean opseq')
    # create_vuln_opseq_files() # should only run the first time this code is ran
    # print('Created vuln opseq')
    vuln, clean = read_dataset()

    dataset, train_set, val_set, test_set = split_dataset(vuln, clean)
    
#     # test that the code is actually working by breaking the validation set - accuracy should be around 50%
#     for (idx, outer_list) in enumerate(val_set):
#         random.shuffle(outer_list[0])
        
    trainLoader, valLoader, testLoader = createLoaders(train_set, val_set, test_set)
#     testRealisticLoader = testRealisticSet()
    testDetectionLoader = create_detection_set()
    net = VulnerabilityDetectorNetwork()
    # net.to(device)
    print(net)
    print('started training network')
    before = time.time()
    
    net, epochs_list, train_acc_list, val_acc_list, loss_list = train_network(net, trainLoader, valLoader)
    print('started testing network')
    precision, recall, f_score, classification_accuracy = test_network(net, testLoader)
    print('test set',"{0:.6f}".format(precision),"{0:.6f}".format(recall),"{0:.6f}".format(f_score),"{0:.6f}".format(classification_accuracy))
    print()
    
#     precision, recall, f_score, classification_accuracy = test_network(net, testRealisticLoader)
#     print('test set',"{0:.6f}".format(precision),"{0:.6f}".format(recall),"{0:.6f}".format(f_score),"{0:.6f}".format(classification_accuracy))
#     print()
    test_detection(net, testDetectionLoader)
    eval_time = time.time() - before
    print ("Time to run: ", eval_time)
    drawEpochsAccuracyGraph(epochs_list, train_acc_list, val_acc_list)
    drawEpochsLossGraph(epochs_list, loss_list)

In [0]:
# print('Running Test Case: Evaluation Function')
# test_case_result_eval_function = test_cases.test_evaluation_function(evaluate_network_performance)
# print('Test Case Result: ', test_case_result_eval_function)
# print()

# print('Running Test Case: Splitting Dataset')
# test_vuln,test_clean = read_dataset()
# test_case_result_split_dataset = test_cases.test_split_dataset(split_dataset, test_vuln, test_clean)
# print('Test Case Result: ', test_case_result_split_dataset)
# print()

# print('Running Test Case: Neural Network')
# test_case_result_network = test_cases.test_network(VulnerabilityDetectorNetwork)
# print('Test Case Result: ', test_case_result_network)
# print()

# if test_case_result_eval_function and test_case_result_split_dataset and test_case_result_network:
  main()
# else:
#     print('one or more test cases failed - quitting')
#     quit()

VulnerabilityDetectorNetwork(
  (emb1): Embedding(205, 8)
  (conv1): Conv2d(1, 64, kernel_size=(8, 8), stride=(1, 1))
  (lin1): Linear(in_features=64, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=2, bias=True)
  (drop_out): Dropout(p=0.5)
)
started training network


In [0]:
# from __future__ import print_function, with_statement, division
# import copy
# import os
# import torch
# from tqdm.autonotebook import tqdm
# from torch.optim.lr_scheduler import _LRScheduler
# import matplotlib.pyplot as plt
# import pdb


# class LRFinder(object):
#     """Learning rate range test.

#     The learning rate range test increases the learning rate in a pre-training run
#     between two boundaries in a linear or exponential manner. It provides valuable
#     information on how well the network can be trained over a range of learning rates
#     and what is the optimal learning rate.

#     Arguments:
#         model (torch.nn.Module): wrapped model.
#         optimizer (torch.optim.Optimizer): wrapped optimizer where the defined learning
#             is assumed to be the lower boundary of the range test.
#         criterion (torch.nn.Module): wrapped loss function.
#         device (str or torch.device, optional): a string ("cpu" or "cuda") with an
#             optional ordinal for the device type (e.g. "cuda:X", where is the ordinal).
#             Alternatively, can be an object representing the device on which the
#             computation will take place. Default: None, uses the same device as `model`.
#         memory_cache (boolean): if this flag is set to True, `state_dict` of model and
#             optimizer will be cached in memory. Otherwise, they will be saved to files
#             under the `cache_dir`.
#         cache_dir (string): path for storing temporary files. If no path is specified,
#             system-wide temporary directory is used.
#             Notice that this parameter will be ignored if `memory_cache` is True.

#     Example:
#         >>> lr_finder = LRFinder(net, optimizer, criterion, device="cuda")
#         >>> lr_finder.range_test(dataloader, end_lr=100, num_iter=100)

#     Cyclical Learning Rates for Training Neural Networks: https://arxiv.org/abs/1506.01186
#     fastai/lr_find: https://github.com/fastai/fastai

#     """

#     def __init__(self, model, optimizer, criterion, device=None, memory_cache=True, cache_dir=None):
#         self.model = model
#         self.optimizer = optimizer
#         self.criterion = criterion
#         self.history = {"lr": [], "loss": []}
#         self.best_loss = None
#         self.memory_cache = memory_cache
#         self.cache_dir = cache_dir

#         # Save the original state of the model and optimizer so they can be restored if
#         # needed
#         self.model_device = next(self.model.parameters()).device
#         self.state_cacher = StateCacher(memory_cache, cache_dir=cache_dir)
#         self.state_cacher.store('model', self.model.state_dict())
#         self.state_cacher.store('optimizer', self.optimizer.state_dict())

#         # If device is None, use the same as the model
#         if device:
#             self.device = device
#         else:
#             self.device = self.model_device

#     def reset(self):
#         """Restores the model and optimizer to their initial states."""
#         self.model.load_state_dict(self.state_cacher.retrieve('model'))
#         self.optimizer.load_state_dict(self.state_cacher.retrieve('optimizer'))
#         self.model.to(self.model_device)

#     def range_test(
#         self,
#         train_loader,
#         val_loader=None,
#         end_lr=10,
#         num_iter=100,
#         step_mode="exp",
#         smooth_f=0.05,
#         diverge_th=5,
#     ):
#         """Performs the learning rate range test.

#         Arguments:
#             train_loader (torch.utils.data.DataLoader): the training set data laoder.
#             val_loader (torch.utils.data.DataLoader, optional): if `None` the range test
#                 will only use the training loss. When given a data loader, the model is
#                 evaluated after each iteration on that dataset and the evaluation loss
#                 is used. Note that in this mode the test takes significantly longer but
#                 generally produces more precise results. Default: None.
#             end_lr (float, optional): the maximum learning rate to test. Default: 10.
#             num_iter (int, optional): the number of iterations over which the test
#                 occurs. Default: 100.
#             step_mode (str, optional): one of the available learning rate policies,
#                 linear or exponential ("linear", "exp"). Default: "exp".
#             smooth_f (float, optional): the loss smoothing factor within the [0, 1[
#                 interval. Disabled if set to 0, otherwise the loss is smoothed using
#                 exponential smoothing. Default: 0.05.
#             diverge_th (int, optional): the test is stopped when the loss surpasses the
#                 threshold:  diverge_th * best_loss. Default: 5.

#         """
#         # Reset test results
#         self.history = {"lr": [], "loss": []}
#         self.best_loss = None

#         # Move the model to the proper device
#         self.model.to(self.device)

#         # Initialize the proper learning rate policy
#         if step_mode.lower() == "exp":
#             lr_schedule = ExponentialLR(self.optimizer, end_lr, num_iter)
#         elif step_mode.lower() == "linear":
#             lr_schedule = LinearLR(self.optimizer, end_lr, num_iter)
#         else:
#             raise ValueError("expected one of (exp, linear), got {}".format(step_mode))

#         if smooth_f < 0 or smooth_f >= 1:
#             raise ValueError("smooth_f is outside the range [0, 1[")

#         # Create an iterator to get data batch by batch
#         iterator = iter(train_loader)
#         for iteration in tqdm(range(num_iter)):
#             # Get a new set of inputs and labels
#             try:
#                 inputs, labels = next(iterator)				
				
#             except StopIteration:
#                 iterator = iter(train_loader)
#                 inputs, labels = next(iterator)

#             # Train on batch and retrieve loss
#             loss = self._train_batch(inputs, labels)
#             if val_loader:
#                 loss = self._validate(val_loader)

#             # Update the learning rate
#             lr_schedule.step()
#             self.history["lr"].append(lr_schedule.get_lr()[0])

#             # Track the best loss and smooth it if smooth_f is specified
#             if iteration == 0:
#                 self.best_loss = loss
#             else:
#                 if smooth_f > 0:
#                     loss = smooth_f * loss + (1 - smooth_f) * self.history["loss"][-1]
#                 if loss < self.best_loss:
#                     self.best_loss = loss

#             # Check if the loss has diverged; if it has, stop the test
#             self.history["loss"].append(loss)
#             if loss > diverge_th * self.best_loss:
#                 print("Stopping early, the loss has diverged")
#                 break

#         print("Learning rate search finished. See the graph with {finder_name}.plot()")

#     def _train_batch(self, inputs, labels):
#         # Set model to training mode
#         self.model.train()
# # 		inputs_tensor = inputs.type(torch.LongTensor)
# # 		inputs = torch.stack(inputs_tensor).flatten()
# # 		labels = labels.type(torch.LongTensor)
# #         pdb.set_trace()
        
#         # Forward pass
#         self.optimizer.zero_grad()
#         inputs = torch.stack(inputs).flatten()
#         inputs = inputs.type(torch.LongTensor)
#         # Move data to the correct device
#         inputs = inputs.to(self.device)
#         labels = labels.to(self.device)

#         outputs = self.model(inputs.view(-1, 1000))
#         loss = self.criterion(outputs, labels)

#         # Backward pass
#         loss.backward()
#         self.optimizer.step()

#         return loss.item()

#     def _validate(self, dataloader):
#         # Set model to evaluation mode and disable gradient computation
#         running_loss = 0
#         self.model.eval()
#         with torch.no_grad():
#             for inputs, labels in dataloader:
#                 inputs = torch.stack(inputs).flatten()
#                 inputs = inputs.type(torch.LongTensor)
#                 # Move data to the correct device
#                 inputs = inputs.to(self.device)
#                 labels = labels.to(self.device)

#                 # Forward pass and loss computation
#                 outputs = self.model(inputs.view(-1, 1000))
#                 loss = self.criterion(outputs, labels)
#                 running_loss += loss.item() * inputs.size(0)

#         return running_loss / len(dataloader.dataset)

#     def plot(self, skip_start=10, skip_end=5, log_lr=True):
#         """Plots the learning rate range test.

#         Arguments:
#             skip_start (int, optional): number of batches to trim from the start.
#                 Default: 10.
#             skip_end (int, optional): number of batches to trim from the start.
#                 Default: 5.
#             log_lr (bool, optional): True to plot the learning rate in a logarithmic
#                 scale; otherwise, plotted in a linear scale. Default: True.

#         """

#         if skip_start < 0:
#             raise ValueError("skip_start cannot be negative")
#         if skip_end < 0:
#             raise ValueError("skip_end cannot be negative")

#         # Get the data to plot from the history dictionary. Also, handle skip_end=0
#         # properly so the behaviour is the expected
#         lrs = self.history["lr"]
#         losses = self.history["loss"]
#         if skip_end == 0:
#             lrs = lrs[skip_start:]
#             losses = losses[skip_start:]
#         else:
#             lrs = lrs[skip_start:-skip_end]
#             losses = losses[skip_start:-skip_end]

#         # Plot loss as a function of the learning rate
#         plt.plot(lrs, losses)
#         if log_lr:
#             plt.xscale("log")
#         plt.xlabel("Learning rate")
#         plt.ylabel("Loss")
#         plt.show()


# class LinearLR(_LRScheduler):
#     """Linearly increases the learning rate between two boundaries over a number of
#     iterations.

#     Arguments:
#         optimizer (torch.optim.Optimizer): wrapped optimizer.
#         end_lr (float, optional): the initial learning rate which is the lower
#             boundary of the test. Default: 10.
#         num_iter (int, optional): the number of iterations over which the test
#             occurs. Default: 100.
#         last_epoch (int): the index of last epoch. Default: -1.

#     """

#     def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
#         self.end_lr = end_lr
#         self.num_iter = num_iter
#         super(LinearLR, self).__init__(optimizer, last_epoch)

#     def get_lr(self):
#         curr_iter = self.last_epoch + 1
#         r = curr_iter / self.num_iter
#         return [base_lr + r * (self.end_lr - base_lr) for base_lr in self.base_lrs]


# class ExponentialLR(_LRScheduler):
#     """Exponentially increases the learning rate between two boundaries over a number of
#     iterations.

#     Arguments:
#         optimizer (torch.optim.Optimizer): wrapped optimizer.
#         end_lr (float, optional): the initial learning rate which is the lower
#             boundary of the test. Default: 10.
#         num_iter (int, optional): the number of iterations over which the test
#             occurs. Default: 100.
#         last_epoch (int): the index of last epoch. Default: -1.

#     """

#     def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
#         self.end_lr = end_lr
#         self.num_iter = num_iter
#         super(ExponentialLR, self).__init__(optimizer, last_epoch)

#     def get_lr(self):
#         curr_iter = self.last_epoch + 1
#         r = curr_iter / self.num_iter
#         return [base_lr * (self.end_lr / base_lr) ** r for base_lr in self.base_lrs]


# class StateCacher(object):
#     def __init__(self, in_memory, cache_dir=None):
#         self.in_memory = in_memory
#         self.cache_dir = cache_dir

#         if self.cache_dir is None:
#             import tempfile
#             self.cache_dir = tempfile.gettempdir()
#         else:
#             if not os.path.isdir(self.cache_dir):
#                 raise ValueError('Given `cache_dir` is not a valid directory.')

#         self.cached = {}

#     def store(self, key, state_dict):
#         if self.in_memory:
#             self.cached.update({key: copy.deepcopy(state_dict)})
#         else:
#             fn = os.path.join(self.cache_dir, 'state_{}_{}.pt'.format(key, id(self)))
#             self.cached.update({key: fn})
#             torch.save(state_dict, fn)

#     def retrieve(self, key):
#         if key not in self.cached:
#             raise KeyError('Target {} was not cached.'.format(key))

#         if self.in_memory:
#             return self.cached.get(key)
#         else:
#             fn = self.cached.get(key)
#             if not os.path.exists(fn):
#                 raise RuntimeError('Failed to load state in {}. File does not exist anymore.'.format(fn))
#             state_dict = torch.load(fn, map_location=lambda storage, location: storage)
#             return state_dict

#     def __del__(self):
#         """Check whether there are unused cached files existing in `cache_dir` before
#         this instance being destroyed."""
#         if self.in_memory:
#             return

#         for k in self.cached:
#             if os.path.exists(self.cached[k]):
#                 os.remove(self.cached[k])


In [0]:
# vuln, clean = read_dataset()

# dataset, train_set, val_set, test_set = split_dataset(vuln, clean)
# trainLoader, valLoader, testLoader = createLoaders(train_set, val_set, test_set)
# net = VulnerabilityDetectorNetwork()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=1e-6)
# lr_finder = LRFinder(net, optimizer, criterion)
# lr_finder.range_test(trainLoader, end_lr=1, num_iter=100, step_mode="exp")


In [0]:
# lr_finder.plot()

In [0]:
# lr_finder.reset()

In [0]:
# lr_finder.range_test(trainLoader, val_loader=valLoader, end_lr=1, num_iter=100, step_mode="exp")

In [0]:
# lr_finder.plot(skip_end=0)

In [0]:
# lr_finder.reset()